In [ ]:
import math
import numpy as np
import numpy.random as npr

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
class Bacterium:
    next_id = 0
    
    def __init__(self, xy, th, vel, eta, r0, grid_x, grid_y):
        self.position = xy
        self.orientation = th
        self.vel = vel
        self.noise = eta
        self.radius = r0
        Bacterium.next_id += 1
        self.id = Bacterium.next_id
        self.LatticePosition(grid_x, grid_y)
        self.neighbour = list()
        
    def LatticePosition(self, gxx, gyy):
        self.Gx = np.searchsorted(gxx, self.position[0], side = 'right')
        self.Gy = np.searchsorted(gyy, self.position[1], side = 'right')

    def NeighboursFinder(self, OParticlesD, Xg, Yg):
        self.neighbour = list()
                
        self.neighbour = [jj.id for jj in OParticlesD.values() if
                          (self.Gx - 1 <= jj.Gx <= self.Gx + 1) and
                          (self.Gy - 1 <= jj.Gy <= self.Gy + 1) and
                          np.linalg.norm(self.position - jj.position) <= self.radius
                         ]
    def UpdatePosition(self, OParticlesD, SizeX, SizeY, bcx, bcy):
        if len(self.neighbour) > 1:
            oo = [OParticlesD[j].orientation for j in self.neighbour]
            self.orientation = math.atan2(np.mean(np.sin(oo)), 
                                          np.mean(np.cos(oo))
                                         )
        self.orientation += self.noise * npr.uniform(-np.pi, np.pi)
        
        self.position[0] += self.vel * np.cos(self.orientation)
        self.position[1] += self.vel * np.sin(self.orientation)

        if bcx == 'periodic':
            if self.position[0] > SizeX:
                self.position[0] -= SizeX
            elif self.position[0] < 0:
                self.position[0] += SizeX
        elif bcy == 'reflective':
            if self.position[0] > SizeY:
                self.position[0] = 2 * SizeX - self.position[0]
                self.orientation = np.pi - self.orientation
        elif self.position[0] < 0:
                self.position[0] = -self.position[0]
                self.orientation = np.pi - self.orientation
        else:
            raise ValueError('Wrong boundary conditions on x')

        if bcy == 'periodic':
            if self.position[1] > SizeY:
                self.position[1] -= SizeY
            elif self.position[1] < 0:
                self.position[1] += SizeY
        elif bcy == 'reflective':
            if self.position[1] > SizeY:
                self.position[1] = 2 * SizeY - self.position[1]
                self.orientation = -self.orientation
            elif self.position[1] < 0:
                self.position[1] = -self.position[1]
                self.orientation = -self.orientation
        else:
            raise ValueError('Wrong boundary conditions on y')       

class Environment:
    def __init__(self, SizeX, SizeY, int_radius, rho, timestep, timesimulation, BC_x, BC_y):
        self.Lx = SizeX
        self.Ly = SizeY
        self.radius = int_radius
        self.GridX = np.arange(0, self.Lx + 0.1, self.radius)
        self.GridY = np.arange(0, self.Ly + 0.1, self.radius)
        self.N = rho * self.Lx * self.Ly
        self.dt = timestep
        self.tSim = timesimulation
        self.bcX = BC_x
        self.bcY = BC_y
        
def ParticleGeneration(env_obj, vel0, noise):
    Bacterium.next_id = 0
    tPosition = np.column_stack((env_obj.Lx * npr.uniform(size = (env_obj.N, 1)), 
                                 env_obj.Ly * npr.uniform(size = (env_obj.N, 1)))
                               )
    tTheta = 2 * np.pi * npr.uniform(size = (env_obj.N, 1))
    dic = {}
    for x in range(1, env_obj.N + 1):
        xx0 = tPosition[x - 1, :]
        th0 = tTheta[x - 1]
        dic[x] = Bacterium(xy = xx0, 
                           th = float(th0), 
                           vel = vel0, 
                           eta = noise, 
                           r0 = env_obj.radius, 
                           grid_x = env_obj.GridX, 
                           grid_y = env_obj.GridY
                          )
    return dic

In [ ]:
EE = Environment(SizeX = 16,              # Environment's width
                 SizeY = 4,               # Environment's height
                 int_radius = 1,          # Interaction radius
                 rho = 1,                 # Particles' density
                 timestep = 1,            # dt
                 timesimulation = 10000,  # Simulation duration
                 BC_x = 'periodic',       # Either 'periodic' or 'reflective'
                 BC_y = 'reflective',     # Either 'periodic' or 'reflective'
                )

Container = ParticleGeneration(env_obj = EE,  # Environment object
                               vel0 = 0.05,   # Particle velocity
                               noise = 0.2,   # Noise
                              )

# Override

pp = np.array([jj.position for jj in Container.values()])
oo = np.array([jj.orientation for jj in Container.values()])

fig, ax = plt.subplots(figsize=(14, 2))
ax.set_xlim(0, EE.Lx)
ax.set_ylim(0, EE.Ly)
#ax.set_xticks(EE.GridX)
#ax.set_yticks(EE.GridY)
plt.grid()
scatter = ax.scatter(pp[:, 0], pp[:, 1], c = oo, cmap = 'hsv', vmin = 0, vmax = 2 * np.pi)


def animate(i):
    
    global EE, Container #, pCont
    
    for k in range(1, EE.N + 1):
        Container[k].LatticePosition(EE.GridX, EE.GridY)
        
    for j in range(1, EE.N + 1):        
        Container[j].NeighboursFinder(Container, EE.GridX, EE.GridY)
        Container[j].UpdatePosition(Container, EE.Lx, EE.Ly, EE.bcX, EE.bcY)
        
    pp = np.array([jj.position for jj in Container.values()])
    oo = np.array([jj.orientation for jj in Container.values()])

    scatter.set_offsets(pp)
    scatter.set_array(oo)

    return scatter,

anim = FuncAnimation(fig, animate, np.arange(1, EE.tSim), interval = EE.dt, blit = True)
plt.show()
